In [ ]:
# Import all librarires
import os
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted
from collections import defaultdict
import cv2
from scipy.stats import linregress
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

- Set location of images and environmental conditions

In [ ]:
# Set the location of the images.
Drive ="C:/"
Folder = "YOUR FOLDER"
Mearsurements = "YOUR SUB-FOLDER"

# Define the temperature (celsius) and pressure (mbar) for this calibration run
current_temperature = 20
current_pressure = 1005

# Part1 - Data preparation and inspection.
Overview:
- Load RAW images
- Set a pixel threshold for the images
- Crop images (Not advised for coulmn by coulmn calibration)
- Inspect image histograms
- Split images in RGB color channels and save as image stacks (replicate images are averraged)
- Calculate mean intensity, standard deviation, red/green ratio for all pixels in images in the stacks + calculate mean red/green ratio for all individual columns
- Save images with red/green ratio as pixel values
- Insepct Ratio images for uneven distribution caused by uneven illumination by the LED (corrected with column by column calibration approach)
- Optionally smooth the mean ratio column by column to reduce influence of artifacts on the optode during calibration 
- Save dataframe with mean red/green ratio column by column for all calibration data 
---


- Load RAW images and set minimum threshold pixel value for all images

In [ ]:
# Set the path to the input directory
input_dir = f'{Drive}/{Folder}/{Mearsurements}/'

# Get a list of all image files in the input directory
input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)

# Define the threshold value
threshold_value = 256

# Create a list to store the thresholded images
thresholded_images = []

# Calculate the number of rows and columns for the grid
num_images = len(input_files)
num_rows = int(num_images ** 0.5)
num_cols = (num_images + num_rows - 1) // num_rows

# Create the grid of subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))

# Iterate over the image files and display them in the grid
for i, filename in enumerate(input_files):
    # Load the image
    image = tifffile.imread(os.path.join(input_dir, filename))

    # Apply the threshold
    thresholded_image = np.where(image > threshold_value, image, 0)
    thresholded_image_nan = np.where(thresholded_image == 0, np.nan, thresholded_image)

    thresholded_images.append(thresholded_image_nan)

    # Determine the subplot indices
    row_idx = i // num_cols
    col_idx = i % num_cols

    # Display the thresholded image in the corresponding subplot
    axes[row_idx, col_idx].imshow(thresholded_image, cmap='viridis', vmin=256,vmax=4065)
    axes[row_idx, col_idx].set_title(filename[15:])
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Display the grid of images
plt.show()


- Save thresholded images in new directory and display one example image to check crop settings.

In [ ]:
# Set output directory
output_dir = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save Thresholded images
for i, filename in enumerate(input_files):
    output_path = os.path.join(output_dir, filename)
    tifffile.imwrite(output_path, thresholded_images[i])
    
# Set the path to the directory containing the thresholded images
thresholded_dir = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/'

# Get a list of all thresholded image files in the directory
thresholded_files = [f for f in os.listdir(thresholded_dir) if f.endswith('.tiff')]
thresholded_files = natsorted(thresholded_files)

test_thresholded_image = tifffile.imread(os.path.join(thresholded_dir,thresholded_files[0]))
# Fullsize crop setting[0:3040,0:4056] 
test_thresholded_image = test_thresholded_image[0:3040,0:4056] 
plt.imshow(test_thresholded_image)

- Show histogram of Red, Green, and Blue color channel for each image in the directory.

In [ ]:
# Calculate the number of rows and columns for the grid
num_images = len(thresholded_files)
num_rows = int(num_images ** 0.5)
num_cols = (num_images + num_rows - 1) // num_rows

# Create the grid of subplots for histograms
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 8))

# Iterate over the thresholded image files and display histograms in the grid
for i, filename in enumerate(thresholded_files):
    # Load the thresholded image
    thresholded_image = tifffile.imread(os.path.join(thresholded_dir, filename))

    # Set Crop dimension (OPTIONAL)
    thresholded_image =thresholded_image[0:3040,0:4056] 

    # Replace NaN values with 0
    thresholded_image[np.isnan(thresholded_image)] = 0

    # Get the color channels in Bayer order (BGGR)
    red = thresholded_image[1::2, 1::2]
    green1 = thresholded_image[0::2, 1::2]
    green2 = thresholded_image[1::2, 0::2]
    green = np.add(green1, green2) / 2
    blue = thresholded_image[0::2, 0::2]

    # Calculate the minimum and maximum value of the dataset
    min_value_red = np.min(red)
    min_value_green = np.min(green)
    min_value_blue = np.min(blue)
    max_value_red = np.max(red)
    max_value_green = np.max(green)
    max_value_blue = np.max(blue)
    min_value = min(min_value_red, min_value_green, min_value_blue)
    max_value = max(max_value_red, max_value_green, max_value_blue)

   # Calculate and plot the histograms for each color channel
    if np.isfinite(min_value) and np.isfinite(max_value):
        histogram, bin_edges = np.histogram(red, bins=4095, range=(threshold_value, max_value))
        axes[i // num_cols, i % num_cols].plot(bin_edges[0:-1], histogram, color='red', linewidth=1)

        histogram, bin_edges = np.histogram(green, bins=4095, range=(threshold_value, max_value))
        axes[i // num_cols, i % num_cols].plot(bin_edges[0:-1], histogram, color='green', linewidth=1)

        histogram, bin_edges = np.histogram(blue, bins=4095, range=(threshold_value, max_value))
        axes[i // num_cols, i % num_cols].plot(bin_edges[0:-1], histogram, color='blue', linewidth=1)

    # Set the title of the subplot as the filename
    axes[i // num_cols, i % num_cols].set_title(filename[15:])

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=1.5, wspace=1)
plt.show()


- Create and save image stacks of each color channel.
- Stack averages are made for replicates. 

In [ ]:
input_dir = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/'
output_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Stacks'

if not os.path.exists(output_path):
    os.makedirs(output_path)

input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)

# Group files by the identifying number
groups = defaultdict(list)
for file in input_files:
    numeric_num = float(os.path.splitext(file)[0][24:].replace(",", "."))
    groups[numeric_num].append(file)

# Initialize lists to store the averaged channel stacks
averaged_red_stack = []
averaged_green_stack = []
averaged_blue_stack = []

for numeric_num, files in groups.items():
    red_accumulator = []
    green_accumulator = []
    blue_accumulator = []

    # Load and accumulate each replicate image
    for file in files:
        img = tifffile.imread(os.path.join(input_dir, file))

         ### Set Crop dimension (OPTIONAL) ### No crop setting: [0:3040,0:4056]
        img = img[0:3040,0:4056]  
        
        red_channel = img[1::2, 1::2]
        blue_channel = img[0::2, 0::2]
        green_channel_1 = img[0::2, 1::2]
        green_channel_2 = img[1::2, 0::2]
        green_channel = np.add(green_channel_1, green_channel_2) // 2

        red_accumulator.append(red_channel)
        blue_accumulator.append(blue_channel)
        green_accumulator.append(green_channel)
    
    # Calculate the average of replicates for each channel
    red_avg = np.mean(red_accumulator, axis=0)
    green_avg = np.mean(green_accumulator, axis=0)
    blue_avg = np.mean(blue_accumulator, axis=0)
    
    # Add the averaged channels to the corresponding stacks
    averaged_red_stack.append(red_avg)
    averaged_green_stack.append(green_avg)
    averaged_blue_stack.append(blue_avg)

# Stack the averaged channels along the third axis
averaged_red_stack = np.stack(averaged_red_stack, axis=0)
averaged_green_stack = np.stack(averaged_green_stack, axis=0)
averaged_blue_stack = np.stack(averaged_blue_stack, axis=0)

# Save the averaged stacks to separate TIFF files
tifffile.imwrite(os.path.join(output_path, "averaged_stacked_red_channel.tiff"), averaged_red_stack)
tifffile.imwrite(os.path.join(output_path, "averaged_stacked_green_channel.tiff"), averaged_green_stack)
tifffile.imwrite(os.path.join(output_path, "averaged_stacked_blue_channel.tiff"), averaged_blue_stack)



- Load image stacks

In [ ]:
output_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Stacks'
# Get a list of all TIFF files in the directory
files = [f for f in os.listdir(output_path) if f.endswith('.tiff')]
print(files)

green_stack = tifffile.imread(os.path.join(output_path, files[1]))
red_stack = tifffile.imread(os.path.join(output_path, files[2]))
blue_stack = tifffile.imread(os.path.join(output_path, files[0]))

#Get length of image stacks
num_images_green = len(green_stack)
num_images_red = len(red_stack)
num_images_blue = len(blue_stack)
if num_images_green == num_images_red and num_images_blue:
    print("Equal stacks")    
else:
    print("unequal stacks")

num_images = num_images_green
num_columns = green_stack[0].shape[1]# Define the number of columns per image

print("number of images in stack:",num_images)
print("number of columns:", num_columns)

- Get Red/Green intensity ratio and save ratio images in new directory

In [ ]:
output_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Ratio_Images/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

mean_red_intensity =[]
mean_green_intensity = []
mean_blue_intensity = []
std_red_intensity = []
std_green_intensity = []
std_blue_intensity =[]

# Initialize lists to store column-wise means and stds for the red/green ratio
mean_red_green_ratio_per_column = []
std_red_green_ratio_per_column = []

for i in range(num_images):
    mean_red_intensity.append(np.mean(red_stack[i]))
    mean_green_intensity.append(np.mean(green_stack[i]))
    mean_blue_intensity.append(np.mean(blue_stack[i]))
    std_red_intensity.append(np.std(red_stack[i]))
    std_green_intensity.append(np.std(green_stack[i]))
    std_blue_intensity.append(np.std(blue_stack[i]))

    # Calculate the intensity ratio for each pixel
    ratio = red_stack[i] / green_stack[i]

    # Construct the output file path
    output_file = os.path.join(output_path, f"ratio_image_{i}.tiff")
    # Save the new image
    tifffile.imwrite(output_file, ratio)

    # Calculate the mean and std of the ratio for each column
    mean_ratio_per_column = np.mean(ratio, axis=0)  # Average across rows for each column
    std_ratio_per_column = np.std(ratio, axis=0)    # Standard deviation across rows for each column

    mean_red_green_ratio_per_column.append(mean_ratio_per_column)
    std_red_green_ratio_per_column.append(std_ratio_per_column)

# Output the column-wise means for the first image as an example
print(mean_red_green_ratio_per_column[0])


- Load two individual ratio images and display ratio histograms for vertially distinct sections in the image to check for uneven illumination patterns. 
- Displays column averages for all images in the plot.

In [ ]:
# Load the first image
image1_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Ratio_Images/ratio_image_0.tiff'
image1 = tifffile.imread(image1_path)

# Load the second image
image2_path = f'{Drive}/{Folder}/{Mearsurements}/Threshold_images/Ratio_Images/ratio_image_10.tiff'
image2 = tifffile.imread(image2_path)

# Number of sections to divide the images into
num_sections = 10
section_width1 = image1.shape[1] // num_sections
section_width2 = image2.shape[1] // num_sections

# Calculate the median for the entire image
median_value_1 = np.median(image1)
median_value_2 = np.median(image2)

# Plotting setup
plt.figure(figsize=(12, 6))
#plt.title('Histograms of Vertical Sections for Two Images', fontsize=14) # Change title size here
plt.xlabel('Red/Green Ratio', fontsize=20) # Change x-axis label size here
plt.ylabel('Frequency', fontsize=20) # Change y-axis label size here
plt.xticks(fontsize=16) # Change x-axis tick size here
plt.yticks(fontsize=16) # Change y-axis tick size here

# Function to plot histograms for each section of an image
def plot_histograms(image, section_width, image_label, add_legend):
    for i in range(num_sections):
        # Extract the section
        section = image[:, i*section_width:(i+1)*section_width]

        # Calculate the histogram for the section
        histogram, bin_edges = np.histogram(section, bins=150, range=[np.min(image),np.max(image)])

        # Plot the histogram
        if add_legend:
            plt.plot(bin_edges[0:-1], histogram, label=f'Section {i+1}')
        else:
            plt.plot(bin_edges[0:-1], histogram)

# Plot histograms for the first image with legend
plot_histograms(image1, section_width1, 'Image 1', True)

# Plot histograms for the second image without adding to legend
plot_histograms(image2, section_width2, 'Image 2', False)
# Add a line for the median
plt.axvline(median_value_1, color='k', linestyle='dashed', linewidth=2, label='Median: 0% Air Sat.')
plt.axvline(median_value_2, color='red', linestyle='dashed', linewidth=2, label='Median: 100% Air Sat.')
plt.legend(fontsize=13)
plt.savefig("Calbration_ratio_distribution.png", dpi=300)
plt.show()

# plot mean ratios per column for each image
for i, mean_ratio in enumerate(mean_red_green_ratio_per_column):
    plt.plot(mean_ratio, label=f'Image {i+1}')

plt.xlabel('Column Index')
plt.ylabel('Mean Red/Green Ratio')
plt.title('Mean Red/Green Ratio per Column Across All Images')
plt.show()


- Displays linear, 2nd degree polynominal and 3rd degree polynomial fits on average coulmn values for all images in the directory. 

In [ ]:

# Initialize dictionaries for storing polynomial coefficients and smoothed data
polynomial_coefficients_linear = {}
polynomial_coefficients_2nd = {}
polynomial_coefficients_3rd = {}
smoothed_data_linear = {}
smoothed_data_2nd = {}
smoothed_data_3rd = {}

# Specify which regression types to plot (can be 'linear', '2nd', '3rd', or any combination)
regression_types_to_plot = ['linear', '2nd', '3rd']  # Adjust this list based on your needs

# Number of images you're working with
num_images = len(mean_red_green_ratio_per_column)

# Calculate the number of columns based on regression types to plot
num_cols = len(regression_types_to_plot)

# Determine the grid size
grid_rows = num_images  # One row per image
grid_cols = num_cols  # Columns based on the number of regression types
fig_width = 15  # Width of the figure, adjust as necessary
fig_height = 5 * num_images  # Height of the figure, adjust based on the number of images

# Create a figure with a specified size
plt.figure(figsize=(fig_width, fig_height))

# Iterate over each image's mean red/green ratio per column
for i, mean_ratio in enumerate(mean_red_green_ratio_per_column):
    column_indices = np.arange(len(mean_ratio))
    
    # Perform polynomial regressions of degrees 1, 2, and 3
    coeffs_linear = np.polyfit(column_indices, mean_ratio, 1)
    coeffs_2nd = np.polyfit(column_indices, mean_ratio, 2)
    coeffs_3rd = np.polyfit(column_indices, mean_ratio, 3)
    
    # Generate polynomial functions from coefficients
    poly_func_linear = np.poly1d(coeffs_linear)
    poly_func_2nd = np.poly1d(coeffs_2nd)
    poly_func_3rd = np.poly1d(coeffs_3rd)
    
    # Calculate fitted values for all regression types
    fitted_vals_linear = poly_func_linear(column_indices)
    fitted_vals_2nd = poly_func_2nd(column_indices)
    fitted_vals_3rd = poly_func_3rd(column_indices)

    # Calculate R^2 scores
    r2_linear = r2_score(mean_ratio, fitted_vals_linear)
    r2_2nd = r2_score(mean_ratio, fitted_vals_2nd)
    r2_3rd = r2_score(mean_ratio, fitted_vals_3rd)

    # Save coefficients, smoothed data, and R^2 scores
    polynomial_coefficients_linear[f'Image {i}'] = coeffs_linear
    polynomial_coefficients_2nd[f'Image {i}'] = coeffs_2nd
    polynomial_coefficients_3rd[f'Image {i}'] = coeffs_3rd

    smoothed_data_linear[f'Image {i}'] = fitted_vals_linear
    smoothed_data_2nd[f'Image {i}'] = fitted_vals_2nd
    smoothed_data_3rd[f'Image {i}'] = fitted_vals_3rd
    
    # Plotting logic for each regression type with R^2 score included in the title
    for j, reg_type in enumerate(regression_types_to_plot):
        ax = plt.subplot(grid_rows, grid_cols, i * num_cols + j + 1)
        ax.plot(column_indices, mean_ratio, 'o', label='Original data')
        
        if reg_type == 'linear':
            ax.plot(column_indices, fitted_vals_linear, 'b-', label='Linear Fit')
            ax.set_title(f'Image {i+1} - Linear, R^2={r2_linear:.4f}')
        elif reg_type == '2nd':
            ax.plot(column_indices, fitted_vals_2nd, 'r-', label='2nd Degree Fit')
            ax.set_title(f'Image {i+1} - 2nd Degree, R^2={r2_2nd:.4f}')
        elif reg_type == '3rd':
            ax.plot(column_indices, fitted_vals_3rd, 'g-', label='3rd Degree Fit')
            ax.set_title(f'Image {i+1} - 3rd Degree, R^2={r2_3rd:.4f}')
        
        ax.legend()

# Adjust layout for better spacing and display the plots
plt.tight_layout()
plt.show()


- Create dataframe for calibration data with RAW data and smoothed data. 

In [ ]:
files_dir = f'{Drive}/{Folder}/{Mearsurements}/' 
files = [f for f in os.listdir(files_dir) if f.endswith('.tiff')]
files = natsorted(files)

# Updated df_ratios to include original and both 2nd and 3rd degree smoothed ratios
if 'df_ratios' not in globals():
    df_ratios = pd.DataFrame(columns=['Image ID', 'Oxygen%', 'Column Index', 'Original Ratio', '2nd Degree Smoothed Ratio',
                                      '3rd Degree Smoothed Ratio', 'Linear Smoothed Ratio', 'Temp', 'Atmospheric Pressure'])

# Set the fixed number of iterations based on number of image sin the stacks
fixed_iterations = num_images 

for i in range(num_images):
    key = f"Image {i}"
    # Ensure we have smoothed data for both polynomial degrees; if not, skip the current iteration
    if key not in smoothed_data_2nd or key not in smoothed_data_3rd:
        print(f"Skipping: {key} not found in smoothed_data for either 2nd or 3rd degree.")
        continue

    file_name = files[i]
    numeric_num = float(os.path.splitext(file_name)[0][24:].replace(",", "."))

    # Original data for the current image
    original_values = mean_red_green_ratio_per_column[i]
    num_columns = len(original_values)

    smoothed_values_2nd = smoothed_data_2nd[key]
    smoothed_values_3rd = smoothed_data_3rd[key]
    smoothed_linear = smoothed_data_linear[key]

    # Prepare data for the current image, including temperature and original + smoothed ratios
    rows_to_add = [{'Image ID': i+1,
                    'Oxygen%': numeric_num,
                    'Column Index': col,
                    'Original Ratio': original_values[col],
                    '2nd Degree Smoothed Ratio': smoothed_values_2nd[col],
                    '3rd Degree Smoothed Ratio': smoothed_values_3rd[col],
                    'Linear Smoothed Ratio': smoothed_linear[col],
                    'Temp': current_temperature,
                    'Atmospheric Pressure':current_pressure} for col in range(num_columns)]

    # Append new rows to df_ratios
    df_ratios = pd.concat([df_ratios, pd.DataFrame(rows_to_add)], ignore_index=True)

- Optionally apply below block to convert air saturation (%) to oxygen partial pressure (hPa) in the dataframe.

In [ ]:
def calculate_partial_pressure_o2(row, solubility_dict):
    # Interpolate or directly use oxygen solubility from the dictionary
    temperatures = np.array(list(solubility_dict.keys()))
    if row['Temp'] in solubility_dict:
        solubility_umol_L = solubility_dict[row['Temp']]
    else:
        # Simple interpolation for temperatures not directly available in the dictionary
        closest_temp = min(temperatures, key=lambda x: abs(x - row['Temp']))
        solubility_umol_L = solubility_dict[closest_temp]

    # Convert solubility from umol/L to mg/L (using oxygen's molar mass, 32.00 g/mol)
    solubility_mg_L = solubility_umol_L * 32.00 / 1000

    # Adjust concentration based on air saturation percentage
    C_O2 = solubility_mg_L * (row['Oxygen%'] / 100.0)

    # Calculate partial pressure of O2 in water based on Henry's Law
    # Here, we use atmospheric pressure and O2 percentage to approximate P_O2 for 100% saturation
    oxygen_percentage_in_air = 0.21
    P_O2_atm = row['Atmospheric Pressure'] * oxygen_percentage_in_air  # Partial pressure of O2 in atmosphere
    P_O2 = P_O2_atm * (row['Oxygen%'] / 100.0)  # Adjust for actual air saturation

    return P_O2

Oxygen_solubility_dict = {0: 456.6, 1: 444.0, 2: 431.9,3: 420.4,4: 409.4,5: 398.9,6: 388.8,7: 379.2,8: 369.9,9: 361.1,10: 352.6,11: 344.4,12: 336.6,13: 329.1,14: 321.9,15: 314.9,16: 308.3,17: 301.8,18: 295.6,19: 289.7,20: 283.9,21: 278.3,22: 273.0,23: 267.8,24: 262.8,25: 257.9,26: 253.2,27: 248.7,28: 244.3,29: 240.0,30: 235.9,31: 231.9,32: 228.0,33: 224.2,34: 220.5,35: 217.0,36: 213.5,37: 210.1,38: 206.7,39: 203.5,40: 200.4}

# Apply the function to calculate partial pressure of O2 in hPa
df_ratios['hPa'] = df_ratios.apply(calculate_partial_pressure_o2, args=(Oxygen_solubility_dict,), axis=1)

df_ratios

<b> - Go back to start and run calibration data with other temperatures to add them to the Dataframe (df_ratios). Save to Excel file when all data is added and continue. </b>

In [ ]:
# Specify the filename and path for the Excel file you want to save
excel_file_path = f'{Drive}/{Folder}/RAW_Calibration_data.xlsx'

# Save the DataFrame as an Excel file
df_ratios.to_excel(excel_file_path, index=False)

# Output the DataFrame (optional, you might want to print a confirmation message instead)
print(f'DataFrame saved as Excel file at: {excel_file_path}')

---